<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>5. Analizar el estado en RNN</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Dataset](#section1)
    * [1.1. Cargar el dataset](#section1.1)
    * [1.2. Secuenciar el probelma](#section1.2)
    * [1.3. Procesamiento de datos](#section1.3)
* [2. LSTM de secuencia de uno a uno carácter](#section2)
* [3. LSTM con ventana](#section3)
    * [3.1. Secuenciar el problema](#section3.1)
    * [3.2. Procesamiento de datos](#section3.2)
    * [3.3. Resultados](#section3.3)
* [4. LSTM ventana de intervalo de tiempo](#section4)
    * [4.1. Secuenciar el problema](#section4.1)
    * [4.2. Procesamiento de datos](#section4.2)
    * [4.3. Resultados](#section4.3)
* [5. LSTM manteniendo stado entre muestras de un batch](#section5)
    * [5.1. Activar secuencialidad en el entrenamient](#section5.1)
    * [5.2. Resultados](#section5.2)
* [6. LSTM con estado para un mapeo de un carácter a un carácter](#section6)
    * [6.1. Definición del problema](#section6.1)
    * [6.2. Procesamiento de datos](#section6.2)
    * [6.3. Diseñar la LSTM](#section6.3)
    * [6.4. Resultados y verificación](#section6.4)
* [7. LSTM con entrada de longitud variable a salida de un solo carácter](#section7)
    * [7.1. Secuenciar el problema](#section7.1)
    * [7.2. Procesamiento de datos](#section7.2)
    * [7.3. Diseñar la LSTM](#section7.3)
    * [7.4. Resultados y verificación](#section7.4)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección, aprenderemos cómo  Keras mantiene el estado en las redes LSTM:
* Desarrollar una red LSTM "ingenua".
* Administrar el estado de LSTM a través de batchs y funciones.
* Administrar manualmente el estado para la predicción con estado.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Procesamiento de datos</font>

En este tutorial vamos a desarrollar y contrastar un problema de predicción de secuencia para aprender el alfabeto, i.e., dada una letra del alfabeto, predecir la siguiente. 

<a id="section1.1"></a>
# <font color="#004D7F" size=5>1.1. Cargar el dataset</font>

En primer lugar, importemos todas las clases, funciones y el dataset del alfabeto que deberemos formatearla a números. 

In [100]:
# Naive LSTM to learn one-char to one-char mapping with all data in each batch
import numpy as np
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import LSTM
from keras.layers import Dropout
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

# define the raw dataset
alfabeto = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# create mapping of characters to integers (0-25) and the reverse
char_to_int= dict((c, i) for i, c in enumerate(alfabeto))
int_to_char= dict((i, c) for i, c in enumerate(alfabeto))


In [101]:
char_to_int

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

<a id="section1.2"></a>
# <font color="#004D7F" size=5>1.2. Secuenciar el probelma</font>

Podemos crear los pares de entrada definiendo una longitud de secuencia de entrada, usaremos una longitud de entrada de 1. 

In [102]:
# prepare the dataset of input to output pairs encoded as integers
seq_long= 1
dataX= []
dataY= []

for i in range(0, len(alfabeto)-seq_long, 1):
  seq_in= alfabeto[i:i+seq_long]
  seq_out= alfabeto[i+seq_long]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out)


A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


<a id="section1.3"></a>
# <font color="#004D7F" size=5>1.3. Procesamiento de datos</font>

Necesitamos remodelar la matriz NumPy en un formato esperado por las redes LSTM, es decir _[muestras, pasos de tiempo, características]._

In [103]:
# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (len(dataX),1,seq_long)) 
X

array([[[ 0]],

       [[ 1]],

       [[ 2]],

       [[ 3]],

       [[ 4]],

       [[ 5]],

       [[ 6]],

       [[ 7]],

       [[ 8]],

       [[ 9]],

       [[10]],

       [[11]],

       [[12]],

       [[13]],

       [[14]],

       [[15]],

       [[16]],

       [[17]],

       [[18]],

       [[19]],

       [[20]],

       [[21]],

       [[22]],

       [[23]],

       [[24]]])

Una vez remodelados, normalizamos los números enteros de entrada al rango de 0 a 1

In [104]:
# normalize
X=X/float(len(alfabeto))
X

array([[[0.        ]],

       [[0.03846154]],

       [[0.07692308]],

       [[0.11538462]],

       [[0.15384615]],

       [[0.19230769]],

       [[0.23076923]],

       [[0.26923077]],

       [[0.30769231]],

       [[0.34615385]],

       [[0.38461538]],

       [[0.42307692]],

       [[0.46153846]],

       [[0.5       ]],

       [[0.53846154]],

       [[0.57692308]],

       [[0.61538462]],

       [[0.65384615]],

       [[0.69230769]],

       [[0.73076923]],

       [[0.76923077]],

       [[0.80769231]],

       [[0.84615385]],

       [[0.88461538]],

       [[0.92307692]]])

Finalmente, En este problema cada una de las 26 letras representa una clase diferente por lo tanto deberemos convertilo con One-Hot Enconding a través de la función `to_categorical()`.

In [105]:
# one hot encode the output variable
y= np_utils.to_categorical(dataY) 
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM de secuencia de uno a uno carácter</font>

Comencemos diseñando un LSTM simple para aprender a predecir el siguiente carácter del alfabeto dado el contexto de un solo carácter. Nuestro modelo será:
1. Una red LSTM con 32 unidades
2. Una capa de salida utilizando la función de activación Softmax para hacer predicciones. 
2. Compilación con `categorical_crossentropy`, adam, 500 épocas y tamaño de batch a 1.

In [106]:
print(X.shape)
print(y.shape)

(25, 1, 1)
(25, 26)


In [107]:
# create and fit the model
model= Sequential()
model.add(LSTM(32,input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 
model.fit(X,y,epochs=500,batch_size=1, verbose=2)

Epoch 1/500
25/25 - 2s - loss: 3.2639 - accuracy: 0.0000e+00
Epoch 2/500
25/25 - 0s - loss: 3.2558 - accuracy: 0.0400
Epoch 3/500
25/25 - 0s - loss: 3.2530 - accuracy: 0.0400
Epoch 4/500
25/25 - 0s - loss: 3.2501 - accuracy: 0.0400
Epoch 5/500
25/25 - 0s - loss: 3.2472 - accuracy: 0.0400
Epoch 6/500
25/25 - 0s - loss: 3.2441 - accuracy: 0.0400
Epoch 7/500
25/25 - 0s - loss: 3.2414 - accuracy: 0.0400
Epoch 8/500
25/25 - 0s - loss: 3.2381 - accuracy: 0.0400
Epoch 9/500
25/25 - 0s - loss: 3.2350 - accuracy: 0.0000e+00
Epoch 10/500
25/25 - 0s - loss: 3.2315 - accuracy: 0.0400
Epoch 11/500
25/25 - 0s - loss: 3.2276 - accuracy: 0.0400
Epoch 12/500
25/25 - 0s - loss: 3.2235 - accuracy: 0.0000e+00
Epoch 13/500
25/25 - 0s - loss: 3.2203 - accuracy: 0.0400
Epoch 14/500
25/25 - 0s - loss: 3.2153 - accuracy: 0.0400
Epoch 15/500
25/25 - 0s - loss: 3.2111 - accuracy: 0.0000e+00
Epoch 16/500
25/25 - 0s - loss: 3.2058 - accuracy: 0.0400
Epoch 17/500
25/25 - 0s - loss: 3.2008 - accuracy: 0.0400
Epoch 1

Evaluamos

In [108]:
# summarize performance of the model
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100)) 

Model Accuracy: 88.00%


Volvemos a los datos originales para ver como han ido las predicciones

In [109]:
# demonstrate some model predictions
for patron in dataX:
  x= np.reshape(patron,(1,1,len(patron)))
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[value] for value in patron]
  print(seq_in,'-->',resultado)

# No va bien...

['A'] --> B
['B'] --> C
['C'] --> D
['D'] --> E
['E'] --> F
['F'] --> G
['G'] --> H
['H'] --> I
['I'] --> J
['J'] --> K
['K'] --> L
['L'] --> M
['M'] --> N
['N'] --> O
['O'] --> P
['P'] --> Q
['Q'] --> R
['R'] --> S
['S'] --> T
['T'] --> U
['U'] --> W
['V'] --> W
['W'] --> Z
['X'] --> Z
['Y'] --> Z


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. LSTM con ventana</font>

Utilizamos el método de ventana para intentar mejorar el aprendizaje.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Secuenciar el problema</font>

Podemos intentar el mismo truco para proporcionar más contexto a la red LSTM. Aquí, aumentamos la longitud de la secuencia de 1 a 3, por ejemplo:

In [110]:
seq_long= 3
dataX= []
dataY= []

for i in range(0, len(alfabeto)-seq_long, 1):
  seq_in= alfabeto[i:i+seq_long]
  seq_out= alfabeto[i+seq_long]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out)

ABC -> D
BCD -> E
CDE -> F
DEF -> G
EFG -> H
FGH -> I
GHI -> J
HIJ -> K
IJK -> L
JKL -> M
KLM -> N
LMN -> O
MNO -> P
NOP -> Q
OPQ -> R
PQR -> S
QRS -> T
RST -> U
STU -> V
TUV -> W
UVW -> X
VWX -> Y
WXY -> Z


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Procesamiento de datos</font>

Al igual que antes necesitamos reformatear la entrada a _[samples, time steps, features],_ normalizar y realizar One-Hot Encoding.

In [111]:
# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (len(dataX),1,seq_long))  

# normalize
X= X/float(len(alfabeto)) 

# one hot encode the output variable
y= np_utils.to_categorical(dataY)  

In [112]:
X.shape

(23, 1, 3)

<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Resultados</font>

El diseño de la red quedaría igual que anteriormente.

In [113]:
# create and fit the model
model= Sequential()
model.add(LSTM(32,input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 
model.fit(X,y,epochs=500,batch_size=1, verbose=2)


Epoch 1/500
23/23 - 2s - loss: 3.2703 - accuracy: 0.0435
Epoch 2/500
23/23 - 0s - loss: 3.2584 - accuracy: 0.0435
Epoch 3/500
23/23 - 0s - loss: 3.2515 - accuracy: 0.0435
Epoch 4/500
23/23 - 0s - loss: 3.2457 - accuracy: 0.0435
Epoch 5/500
23/23 - 0s - loss: 3.2398 - accuracy: 0.0435
Epoch 6/500
23/23 - 0s - loss: 3.2331 - accuracy: 0.0435
Epoch 7/500
23/23 - 0s - loss: 3.2269 - accuracy: 0.0435
Epoch 8/500
23/23 - 0s - loss: 3.2201 - accuracy: 0.0435
Epoch 9/500
23/23 - 0s - loss: 3.2134 - accuracy: 0.0435
Epoch 10/500
23/23 - 0s - loss: 3.2057 - accuracy: 0.0435
Epoch 11/500
23/23 - 0s - loss: 3.1981 - accuracy: 0.0435
Epoch 12/500
23/23 - 0s - loss: 3.1901 - accuracy: 0.0435
Epoch 13/500
23/23 - 0s - loss: 3.1817 - accuracy: 0.0435
Epoch 14/500
23/23 - 0s - loss: 3.1731 - accuracy: 0.0435
Epoch 15/500
23/23 - 0s - loss: 3.1633 - accuracy: 0.0435
Epoch 16/500
23/23 - 0s - loss: 3.1539 - accuracy: 0.0435
Epoch 17/500
23/23 - 0s - loss: 3.1447 - accuracy: 0.0435
Epoch 18/500
23/23 - 0s

 Veamos los resultados:

In [114]:
# summarize performance of the model
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100)) 

# demonstrate some model predictions
for patron in dataX:
  x= np.reshape(patron,(1,1,len(patron)))
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[value] for value in patron]
  print(seq_in,'-->',resultado)


Model Accuracy: 82.61%
['A', 'B', 'C'] --> D
['B', 'C', 'D'] --> E
['C', 'D', 'E'] --> F
['D', 'E', 'F'] --> G
['E', 'F', 'G'] --> H
['F', 'G', 'H'] --> I
['G', 'H', 'I'] --> J
['H', 'I', 'J'] --> K
['I', 'J', 'K'] --> L
['J', 'K', 'L'] --> M
['K', 'L', 'M'] --> N
['L', 'M', 'N'] --> O
['M', 'N', 'O'] --> P
['N', 'O', 'P'] --> Q
['O', 'P', 'Q'] --> R
['P', 'Q', 'R'] --> S
['Q', 'R', 'S'] --> T
['R', 'S', 'T'] --> V
['S', 'T', 'U'] --> V
['T', 'U', 'V'] --> X
['U', 'V', 'W'] --> Z
['V', 'W', 'X'] --> Z
['W', 'X', 'Y'] --> Z


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM con ventana e intervalo de tiempo</font>

En Keras, el uso previsto de los LSTM es proporcionar contexto en forma de pasos de tiempo.

<a id="section4.1"></a>
# <font color="#004D7F" size=5>4.1. Secuenciar el problema</font>

Podemos tomar nuestro primer ejemplo y simplemente cambiar la longitud de la secuencia de 1 a 3.

In [115]:
# prepare the dataset of input to output pairs encoded as integers
seq_long= 3
dataX= []
dataY= []

for i in range(0, len(alfabeto)-seq_long, 1):
  seq_in= alfabeto[i:i+seq_long]
  seq_out= alfabeto[i+seq_long]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out) 


ABC -> D
BCD -> E
CDE -> F
DEF -> G
EFG -> H
FGH -> I
GHI -> J
HIJ -> K
IJK -> L
JKL -> M
KLM -> N
LMN -> O
MNO -> P
NOP -> Q
OPQ -> R
PQR -> S
QRS -> T
RST -> U
STU -> V
TUV -> W
UVW -> X
VWX -> Y
WXY -> Z


<a id="section4.2"></a>
# <font color="#004D7F" size=5>4.2. Procesamiento de datos</font>

Al igual que antes necesitamos reformatear la entrada a _[samples, time steps, features],_ normalizar y realizar One-Hot Encoding.

In [116]:
# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (len(dataX),seq_long,1))  # !! cambio paso de tiempo

# normalize
X= X/float(len(alfabeto)) 

# one hot encode the output variable
y= np_utils.to_categorical(dataY)  


In [117]:
X.shape

(23, 3, 1)

<a id="section4.3"></a>
# <font color="#004D7F" size=5>4.3. Resultados</font>

Creamos nuestro modelo

In [118]:
# create and fit the model
model= Sequential()
model.add(LSTM(32,input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 
model.fit(X,y,epochs=500,batch_size=1, verbose=2)


Epoch 1/500
23/23 - 2s - loss: 3.2707 - accuracy: 0.0000e+00
Epoch 2/500
23/23 - 0s - loss: 3.2578 - accuracy: 0.0000e+00
Epoch 3/500
23/23 - 0s - loss: 3.2493 - accuracy: 0.0435
Epoch 4/500
23/23 - 0s - loss: 3.2428 - accuracy: 0.0435
Epoch 5/500
23/23 - 0s - loss: 3.2352 - accuracy: 0.0435
Epoch 6/500
23/23 - 0s - loss: 3.2278 - accuracy: 0.0435
Epoch 7/500
23/23 - 0s - loss: 3.2206 - accuracy: 0.0435
Epoch 8/500
23/23 - 0s - loss: 3.2127 - accuracy: 0.0435
Epoch 9/500
23/23 - 0s - loss: 3.2040 - accuracy: 0.0435
Epoch 10/500
23/23 - 0s - loss: 3.1941 - accuracy: 0.0435
Epoch 11/500
23/23 - 0s - loss: 3.1841 - accuracy: 0.0435
Epoch 12/500
23/23 - 0s - loss: 3.1708 - accuracy: 0.0435
Epoch 13/500
23/23 - 0s - loss: 3.1575 - accuracy: 0.0435
Epoch 14/500
23/23 - 0s - loss: 3.1408 - accuracy: 0.0435
Epoch 15/500
23/23 - 0s - loss: 3.1241 - accuracy: 0.0435
Epoch 16/500
23/23 - 0s - loss: 3.1072 - accuracy: 0.0435
Epoch 17/500
23/23 - 0s - loss: 3.0861 - accuracy: 0.0435
Epoch 18/500
23

Veamos los resultados

In [119]:
# summarize performance of the model
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100)) 

# demonstrate some model predictions
for patron in dataX:
  x= np.reshape(patron,(1,len(patron),1)) # !! cambio reshape
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[value] for value in patron]
  print(seq_in,'-->',resultado)


Model Accuracy: 100.00%
['A', 'B', 'C'] --> D
['B', 'C', 'D'] --> E
['C', 'D', 'E'] --> F
['D', 'E', 'F'] --> G
['E', 'F', 'G'] --> H
['F', 'G', 'H'] --> I
['G', 'H', 'I'] --> J
['H', 'I', 'J'] --> K
['I', 'J', 'K'] --> L
['J', 'K', 'L'] --> M
['K', 'L', 'M'] --> N
['L', 'M', 'N'] --> O
['M', 'N', 'O'] --> P
['N', 'O', 'P'] --> Q
['O', 'P', 'Q'] --> R
['P', 'Q', 'R'] --> S
['Q', 'R', 'S'] --> T
['R', 'S', 'T'] --> U
['S', 'T', 'U'] --> V
['T', 'U', 'V'] --> W
['U', 'V', 'W'] --> X
['V', 'W', 'X'] --> Y
['W', 'X', 'Y'] --> Z


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. LSTM manteniendo estado entre muestras de un batch</font>

La implementación de LSTM restablece el estado de la red después de cada batch.

<a id="section5.1"></a>
# <font color="#004D7F" size=5>5.1. Activar secuencialidad en el entrenamiento</font>

Para asegurarnos de que los patrones de datos de entrenamiento sean secuenciales, debemos poner `shuffle=False` en la función `fit()`.
```python
    model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
```

<a id="section5.2"></a>
# <font color="#004D7F" size=5>5.2. Resultados</font>

Podemos evaluar tanto la capacidad de la red para realizar predicciones al azar como en secuencia.

In [121]:
# prepare the dataset of input to output pairs encoded as integers
seq_long= 1
dataX= []
dataY= []

for i in range(0, len(alfabeto)-seq_long, 1):
  seq_in= alfabeto[i:i+seq_long]
  seq_out= alfabeto[i+seq_long]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out)  

# convert list of lists to array and pad sequences if needed
X= pad_sequences(dataX, maxlen=seq_long, dtype='float32')

# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (X.shape[0],seq_long,1))  # !! cambio paso de tiempo

# normalize
X= X/float(len(alfabeto)) 

# one hot encode the output variable
y= np_utils.to_categorical(dataY)  


A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [122]:
print(X.shape)
print(y.shape)
print(len(dataX))

(25, 1, 1)
(25, 26)
25


In [123]:
# create and fit the model
model= Sequential()
model.add(LSTM(16,input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 
model.fit(X,y,epochs=5000,batch_size=len(dataX), verbose=2, shuffle=False) # cambio batch c/shuffle con mas epoch


Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 2501/5000
1/1 - 0s - loss: 1.4978 - accuracy: 0.8800
Epoch 2502/5000
1/1 - 0s - loss: 1.4975 - accuracy: 0.8800
Epoch 2503/5000
1/1 - 0s - loss: 1.4972 - accuracy: 0.8800
Epoch 2504/5000
1/1 - 0s - loss: 1.4969 - accuracy: 0.8800
Epoch 2505/5000
1/1 - 0s - loss: 1.4966 - accuracy: 0.8800
Epoch 2506/5000
1/1 - 0s - loss: 1.4964 - accuracy: 0.8800
Epoch 2507/5000
1/1 - 0s - loss: 1.4961 - accuracy: 0.8800
Epoch 2508/5000
1/1 - 0s - loss: 1.4958 - accuracy: 0.8800
Epoch 2509/5000
1/1 - 0s - loss: 1.4955 - accuracy: 0.8800
Epoch 2510/5000
1/1 - 0s - loss: 1.4952 - accuracy: 0.8800
Epoch 2511/5000
1/1 - 0s - loss: 1.4949 - accuracy: 0.8800
Epoch 2512/5000
1/1 - 0s - loss: 1.4946 - accuracy: 0.8800
Epoch 2513/5000
1/1 - 0s - loss: 1.4943 - accuracy: 0.8800
Epoch 2514/5000
1/1 - 0s - loss: 1.4940 - accuracy: 0.8800
Epoch 2515/5000
1/1 - 0s - loss: 1.4937 - accuracy: 0.8800
Epoch 2516/5000
1/1 - 0s - loss: 1.4934 - accuracy: 0.

In [124]:
# summarize performance of the model
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100)) 

Model Accuracy: 100.00%


In [125]:
# demonstrate some model predictions
for patron in dataX:
  x= np.reshape(patron,(1,len(patron),1)) # !! cambio reshape
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[value] for value in patron]
  print(seq_in,'-->',resultado)


['A'] --> B
['B'] --> C
['C'] --> D
['D'] --> E
['E'] --> F
['F'] --> G
['G'] --> H
['H'] --> I
['I'] --> J
['J'] --> K
['K'] --> L
['L'] --> M
['M'] --> N
['N'] --> O
['O'] --> P
['P'] --> Q
['Q'] --> R
['R'] --> S
['S'] --> T
['T'] --> U
['U'] --> V
['V'] --> W
['W'] --> X
['X'] --> Y
['Y'] --> Z


In [126]:
# demonstrate predicting random patterns
print('Evaluacion aleatoria')
for i in range(0,20):
  indice_patron= np.random.randint(len(dataX))
  patron= dataX[indice_patron]
  x= np.reshape(patron,(1,len(patron),1))
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[value] for value in patron]
  print(seq_in,'-->',resultado)  


Evaluacion aleatoria
['K'] --> L
['B'] --> C
['I'] --> J
['A'] --> B
['J'] --> K
['I'] --> J
['D'] --> E
['L'] --> M
['Y'] --> Z
['O'] --> P
['L'] --> M
['D'] --> E
['P'] --> Q
['A'] --> B
['S'] --> T
['K'] --> L
['M'] --> N
['W'] --> X
['T'] --> U
['X'] --> Y


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section6"></a>
# <font color="#004D7F" size=6>6. LSTM con estado para un mapeo de un carácter a un carácter</font>

Idealmente, queremos exponer la red a la secuencia completa y dejar que aprenda las interdependencias, en lugar de definir esas dependencias explícitamente en el marco del problema.

<a id="section6.1"></a>
# <font color="#004D7F" size=5>6.1. Definición del problema</font>

Podemos hacer esto en Keras haciendo que las capas de LSTM tengan estado y restableciendo manualmente el estado de la red al final de la época, que también es el final de la secuencia de entrenamiento.

Para este caso especificamos un tamaño de batch de 1. 
```python
    batch_size = 1
    model.add(LSTM(50, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
```

<a id="section6.2"></a>
# <font color="#004D7F" size=5>6.2. Procesamiento de datos</font>

El procesamiento de datos quedará igual que como lo hemos venido realizando con un tamaño de ventana 1.

In [127]:
# prepare the dataset of input to output pairs encoded as integers
seq_long= 1
dataX= []
dataY= []

for i in range(0, len(alfabeto)-seq_long, 1):
  seq_in= alfabeto[i:i+seq_long]
  seq_out= alfabeto[i+seq_long]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out)   

# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (len(dataX),seq_long,1))  # !! cambio paso de tiempo 

# normalize
X= X/float(len(alfabeto))  

# one hot encode the output variable
y= np_utils.to_categorical(dataY)   


A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [128]:
print(X.shape)

(25, 1, 1)


<a id="section6.3"></a>
# <font color="#004D7F" size=5>6.3. Diseñar LSTM</font>

Una diferencia importante en el entrenamiento del LSTM con estado es que lo entrenamos manualmente una época a la vez y restablecemos el estado después de cada época.

In [129]:
# create and fit the model
batch_size = 1

model= Sequential()
model.add(LSTM(50, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 

for i in range(0,300):
  model.fit(X,y, epochs=1,batch_size=batch_size, verbose=2, shuffle=False) 
  model.reset_states()


25/25 - 2s - loss: 3.2810 - accuracy: 0.0000e+00
25/25 - 0s - loss: 3.2577 - accuracy: 0.0800
25/25 - 0s - loss: 3.2450 - accuracy: 0.0800
25/25 - 0s - loss: 3.2317 - accuracy: 0.0400
25/25 - 0s - loss: 3.2149 - accuracy: 0.0800
25/25 - 0s - loss: 3.1907 - accuracy: 0.0800
25/25 - 0s - loss: 3.1502 - accuracy: 0.0800
25/25 - 0s - loss: 3.0802 - accuracy: 0.0800
25/25 - 0s - loss: 2.9991 - accuracy: 0.0800
25/25 - 0s - loss: 2.9456 - accuracy: 0.0800
25/25 - 0s - loss: 2.9229 - accuracy: 0.0800
25/25 - 0s - loss: 2.9509 - accuracy: 0.2000
25/25 - 0s - loss: 2.9338 - accuracy: 0.2400
25/25 - 0s - loss: 2.8016 - accuracy: 0.2000
25/25 - 0s - loss: 2.7177 - accuracy: 0.2000
25/25 - 0s - loss: 2.6154 - accuracy: 0.2000
25/25 - 0s - loss: 2.5280 - accuracy: 0.2800
25/25 - 0s - loss: 2.4664 - accuracy: 0.2400
25/25 - 0s - loss: 2.3619 - accuracy: 0.2800
25/25 - 0s - loss: 2.3075 - accuracy: 0.2800
25/25 - 0s - loss: 2.1896 - accuracy: 0.2800
25/25 - 0s - loss: 2.1667 - accuracy: 0.2800
25/25 

<a id="section6.4"></a>
# <font color="#004D7F" size=5>6.4. Resultados y verificación</font>

Especificamos el tamaño del batch al evaluar el rendimiento de la red.

In [138]:
# summarize performance of the model
score= model.evaluate(X,y, batch_size=batch_size, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100))  

Model Accuracy: 100.00%


Finalmente, podemos demostrar que la red realmente ha aprendido todo el alfabeto. 

In [148]:
model.reset_states()
semilla= [char_to_int['A']]
print(semilla)
x= np.reshape(semilla,(1,len(semilla),1)) 
print(x)
x= x/float(len(alfabeto))
print(x)
prediccion= model.predict(x,verbose=0) # array
indice= np.argmax(prediccion)  
print(indice)
print(int_to_char[indice])

[0]
[[[0]]]
[[[0.]]]
1
B


In [154]:
# demonstrate some model predictions
model.reset_states()

semilla= [char_to_int[alfabeto[0]]]

for i in range(0, len(alfabeto)-1):
  x= np.reshape(semilla,(1,len(semilla),1)) 
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)  
  print(int_to_char[semilla[0]],'-->',int_to_char[indice]) 
  semilla= [indice]


A --> B
B --> C
C --> D
D --> E
E --> F
F --> G
G --> H
H --> I
I --> J
J --> K
K --> L
L --> M
M --> N
N --> O
O --> P
P --> Q
Q --> R
R --> S
S --> T
T --> U
U --> V
V --> W
W --> X
X --> Y
Y --> Z


También podemos ver si la red puede hacer predicciones a partir de una letra arbitraria.

In [159]:
# demonstrate a random starting point
model.reset_states()

letra= 'Q'
semilla= [char_to_int[letra]]

print('Inicio por letra:',letra)
for i in range(0, 5):
  x= np.reshape(semilla,(1,len(semilla),1)) 
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)  
  print(int_to_char[semilla[0]],'-->',int_to_char[indice]) 
  semilla= [indice]


Inicio por letra: Q
Q --> B
B --> C
C --> D
D --> E
E --> F


In [ ]:
# no sale bien

Esto nos dice que podríamos lograr el mismo efecto con un LSTM sin estado preparando datos de entrenamiento como:
```
        ---a --> b
        --ab --> c
        -abc --> d
        abcd --> e
```


Donde la secuencia de entrada se fija en 25 _(a_ a la _y_ para predecir _z)_ y los patrones tienen un prefijo de relleno con ceros. Finalmente, esto plantea la cuestión de entrenar una red LSTM utilizando secuencias de entrada de longitud variable para predecir el siguiente carácter.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section7"></a>
# <font color="#004D7F" size=6>7. LSTM con entrada de longitud variable a salida de un solo carácter</font>

En esta sección exploramos una variación del LSTM sin estado que aprende subsecuencias aleatorias del alfabeto.

<a id="section7.1"></a>
# <font color="#004D7F" size=5>7.1. Secuenciar el problema</font>

Para simplificar:
1. Definiremos una longitud máxima de secuencia de entrada y la estableceremos en un valor pequeño como 5. 
2. En las extensiones, esto podría establecerse en el alfabeto completo (26) o más si permitimos retroceder hasta el inicio de la secuencia. 
3. También necesitamos definir el número de secuencias aleatorias para crear, en este caso, 1,000. 

In [1]:
# Naive LSTM to learn one-char to one-char mapping with all data in each batch
import numpy as np
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import LSTM
from keras.layers import Dropout
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

# define the raw dataset
alfabeto = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# create mapping of characters to integers (0-25) and the reverse
char_to_int= dict((c, i) for i, c in enumerate(alfabeto))
int_to_char= dict((i, c) for i, c in enumerate(alfabeto))

In [2]:
max_seq= 5
inicio= np.random.randint(len(alfabeto)-2)
fin= np.random.randint(inicio, min(inicio+max_seq,len(alfabeto)-1))
print(inicio,fin)
seq_in= alfabeto[inicio:fin+1]
seq_out= alfabeto[fin+1]
print(seq_in,seq_out)

4 5
EF G


In [3]:
# prepare the dataset of input to output pairs encoded as integers
num_entradas= 1000
max_seq= 5
dataX= []
dataY= []

for i in range(num_entradas):
  inicio= np.random.randint(len(alfabeto)-2)
  fin= np.random.randint(inicio, min(inicio+max_seq,len(alfabeto)-1))
  seq_in= alfabeto[inicio:fin+1]
  seq_out= alfabeto[fin+1]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  print(seq_in,"->",seq_out)  


ABCD -> E
FG -> H
HIJK -> L
RSTUV -> W
JK -> L
ABC -> D
HI -> J
PQR -> S
X -> Y
PQ -> R
WX -> Y
PQRS -> T
RS -> T
R -> S
EFG -> H
UV -> W
TUVW -> X
IJKL -> M
PQ -> R
XY -> Z
D -> E
LMN -> O
HIJK -> L
E -> F
LMNO -> P
U -> V
IJKLM -> N
JKLM -> N
M -> N
QRSTU -> V
PQRST -> U
G -> H
FGH -> I
ABCD -> E
AB -> C
GHIJK -> L
UVW -> X
GHIJ -> K
WX -> Y
STU -> V
QRST -> U
AB -> C
GH -> I
EFGH -> I
P -> Q
LMNOP -> Q
Q -> R
V -> W
M -> N
BC -> D
ST -> U
UVW -> X
PQ -> R
JKL -> M
B -> C
C -> D
WXY -> Z
QRST -> U
JK -> L
STUVW -> X
BC -> D
S -> T
V -> W
GH -> I
CDEF -> G
STUV -> W
DEFG -> H
STUVW -> X
MNOPQ -> R
C -> D
V -> W
KLM -> N
NOP -> Q
LMNO -> P
GHI -> J
J -> K
TUVWX -> Y
WXY -> Z
BCD -> E
E -> F
WX -> Y
ST -> U
VWX -> Y
IJKLM -> N
RS -> T
UV -> W
STUV -> W
JKLM -> N
FGHIJ -> K
V -> W
BCDE -> F
O -> P
S -> T
UVWXY -> Z
CD -> E
JKLM -> N
P -> Q
R -> S
ABCD -> E
FG -> H
QR -> S
IJKL -> M
R -> S
U -> V
XY -> Z
STU -> V
GHI -> J
N -> O
CD -> E
UVWX -> Y
MN -> O
WX -> Y
QRSTU -> V
VW -> X
OPQ -> 

<a id="section7.2"></a>
# <font color="#004D7F" size=5>7.2. Procesamiento de datos</font>

Las secuencias de entrada varían en longitud entre 1 y `max_len` y, por lo tanto, requieren relleno de ceros con la función `pad_sequences()`.

In [4]:
# convert list of lists to array and pad sequences if needed
X= pad_sequences(dataX, maxlen=max_seq, dtype='float32') 

# reshape X to be [samples, time steps, features]
X= np.reshape(X, (X.shape[0],max_seq,1)) 

# normalize
X= X/float(len(alfabeto))  

# one hot encode the output variable
y= np_utils.to_categorical(dataY)   


In [5]:
X.shape

(1000, 5, 1)

<a id="section7.3"></a>
# <font color="#004D7F" size=5>7.3. Diseñar LSTM</font>

El modelo entrenado se evalúa en patrones de entrada seleccionados al azar.

In [6]:
# create and fit the model
batch_size= 1

model= Sequential()
model.add(LSTM(50, input_shape=(X.shape[1],1)))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 

model.fit(X,y, epochs=500,batch_size=batch_size, verbose=2) 

Epoch 1/500
1000/1000 - 4s - loss: 3.0684 - accuracy: 0.0640
Epoch 2/500
1000/1000 - 2s - loss: 2.7130 - accuracy: 0.1260
Epoch 3/500
1000/1000 - 2s - loss: 2.3468 - accuracy: 0.1980
Epoch 4/500
1000/1000 - 2s - loss: 2.1261 - accuracy: 0.2550
Epoch 5/500
1000/1000 - 2s - loss: 1.9620 - accuracy: 0.2970
Epoch 6/500
1000/1000 - 2s - loss: 1.8223 - accuracy: 0.3530
Epoch 7/500
1000/1000 - 2s - loss: 1.7065 - accuracy: 0.3800
Epoch 8/500
1000/1000 - 2s - loss: 1.6102 - accuracy: 0.4410
Epoch 9/500
1000/1000 - 2s - loss: 1.5208 - accuracy: 0.4710
Epoch 10/500
1000/1000 - 2s - loss: 1.4301 - accuracy: 0.5170
Epoch 11/500
1000/1000 - 2s - loss: 1.3721 - accuracy: 0.5410
Epoch 12/500
1000/1000 - 2s - loss: 1.3039 - accuracy: 0.5680
Epoch 13/500
1000/1000 - 2s - loss: 1.2482 - accuracy: 0.5860
Epoch 14/500
1000/1000 - 2s - loss: 1.2028 - accuracy: 0.5940
Epoch 15/500
1000/1000 - 2s - loss: 1.1482 - accuracy: 0.6130
Epoch 16/500
1000/1000 - 2s - loss: 1.1106 - accuracy: 0.6400
Epoch 17/500
1000

<a id="section7.4"></a>
# <font color="#004D7F" size=5>7.4. Resultados y verificación</font>

Veamos los resultados y la verificación de los mismos.

In [8]:
# summarize performance of the model
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100))   

Model Accuracy: 98.80%


In [7]:
# demonstrate some model predictions

for i in range(0, 20):
  indice_patron= np.random.randint(len(dataX))
  patron= dataX[indice_patron]  
  x= pad_sequences([patron], maxlen=max_seq, dtype='float32') 
  x= np.reshape(x,(1,max_seq,1)) 
  x= x/float(len(alfabeto))
  prediccion= model.predict(x,verbose=0) 
  indice= np.argmax(prediccion)  
  resultado= int_to_char[indice]
  sec_in= [int_to_char[value] for value in patron]
  print(sec_in,'-->',resultado) 


['S', 'T', 'U', 'V'] --> W
['U', 'V'] --> W
['K'] --> K
['E', 'F'] --> G
['K', 'L', 'M', 'N'] --> O
['Q', 'R', 'S'] --> T
['L', 'M'] --> N
['C', 'D', 'E', 'F'] --> G
['B'] --> C
['N', 'O', 'P', 'Q', 'R'] --> S
['A', 'B', 'C', 'D', 'E'] --> F
['U'] --> V
['I'] --> J
['O', 'P', 'Q', 'R'] --> S
['W'] --> X
['J', 'K'] --> L
['H', 'I'] --> J
['I', 'J', 'K'] --> L
['S', 'T', 'U', 'V', 'W'] --> X
['H', 'I', 'J', 'K'] --> L


<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>